# Piškorky algoritmus minmax

In [1]:
import numpy as np
import time

In [2]:
class State:
    """ Zachycení stavu partie
    gameplan - dvourozměrné pole 3x3
             - 0 - prázdné poole
             - 1 - X
             - 2 - O
    player - hráč, který je v partii na tahu

    current_player - hráč, který je na tahu v daném stavu při prohledávání stavového prostoru
    depth          - hloubka prohledávání stavového prostoru
    max_depth      - maximální délka prohledávání
    """

    generated = 0

    def __init__(self, gameplan, player, current_player=None, depth=0, max_depth=3):
        self.gameplan = gameplan
        self.player = player
        if current_player is None:
            self.current_player = player
        else:
            self.current_player = current_player
        self.depth = depth
        self.max_depth = max_depth
        
        State.generated += 1

    def terminal_test(self):
        """ Metoda otestuje aktuální stav a vrátí hodnotu, zda je partie dohraná a případně, kdo je vítěz

            0 - není konečný stav
            1 - vítězí 1
            2 - vítězí 2
        """

        # pravidla ukončení partie - horizontální a vertikální trojice
        for i in range(3):
            if np.array_equal(self.gameplan[i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[i], [2, 2, 2]):
                return 2
            if np.array_equal(self.gameplan[:, i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[:, i], [2, 2, 2]):
                return 2

        # pravidla ukončení partie - diagonály
        if np.array_equal(self.gameplan.diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(self.gameplan.diagonal(), [2, 2, 2]):
            return 2
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [2, 2, 2]):
            return 2

        return 0

    def utility(self, result):
        """ Metoda vrací ohodnocení aktuálního stavu partie
            0  - remíza
            1  - vítězí hráč, který je na tahu
            -1 - vítězí protihráč
        """
        if result == 0:
            return 0
        elif result == self.player:
            return 1
        else:
            return -1

    def possible_actions(self):
        """ Metoda vrací list možných akcí pro dany stav partie
            Akce je vyjádřena souřadnicemi prázdného hracího pole.
        """
        possible_actions = []
        # Vyhledání prázdných hracích polí
        for i in range(3):
            for j in range(3):
                if self.gameplan[i][j] == 0:
                    possible_actions.append((i, j))
        return possible_actions

    def expand(self, select_action):
        """ Metoda vytvoří nový stav partie tím, že aplikuje vybranou akci na aktualní stav
            Akce je položení 

            V novém stavu bude na tahu protihráč, ale stav se bude hodnotit z pohledu původního hráče
            Zvýší se i hloubka prohledávaného stavového prostoru
        """
        # Kontrola správnosti zadných souřadnic
        if select_action[0] not in range(3):
            return None
        if select_action[1] not in range(3):
            return None

        # hrací pole musí být volné
        if self.gameplan[select_action[0], select_action[1]] != 0:
            return None
        
        new_array = np.copy(self.gameplan)
        new_array[select_action[0], select_action[1]] = self.current_player
        return State(new_array, 
                     self.player, 
                     self.next_current_player(), 
                     self.depth + 1, 
                     max_depth=self.max_depth)
        

    def minmax(self, strategy="max"):
        if self.depth >= self.max_depth:
            return 0, None  # Omezení hloubky hledání
        """"
        Metoda vybere z možných akcí pro daný stav partie tu, která odpovídá strategii

        stategy - jaká strategie se bude používat pro výběr z možných akcí
        """
        
        
        # kontrola stavu partie, pro ukončenou partii se vrací hodnocení z utility metody
        result = self.terminal_test()
        if result != 0:            
            return self.utility(result), action
        

        # inicializace hodnot pro jednotlivé strategie
        if strategy == "max":
            selected_utilization_value = float('-inf')
            next_strategy = "min"
        else:
            selected_utilization_value = float('inf')
            next_strategy = "max"

        # zjištění možných tahů
        actions = self.possible_actions()

        # vybraný tah se naplní prvním tahem z možných akcí
        selected_action = actions[0]

        # hledání optimálního tahu pro všechny možné tahy
        for action in actions:

            # expanze stavu
            expanded_state = self.expand(action)

            # kontrola konce partie pro expandovaný stav
            result = expanded_state.terminal_test()

            if result != 0:
                # partie končí, vrácení ohodnocení stavu a akce
                return expanded_state.utility(result), action
            else:
                # partie pokračuje
                # pokud v expandovaném stavu existuje alespoň jeden stav, tak stav dále expanduj, jinak je to remíza
                if len(expanded_state.possible_actions()) == 0:
                    utilization = 0
                else:
                    # rekurzivní volání
                    # todo - přidání omezení prohledávání do hloubky
                    utilization, _ = expanded_state.minmax(next_strategy)

                # podle strategie se ohodnocená akce volí jako vybraná akce
                if utilization > selected_utilization_value and strategy == "max":
                    selected_utilization_value = utilization
                    selected_action = action
                elif utilization < selected_utilization_value and strategy == "min":
                    selected_utilization_value = utilization
                    selected_action = action

        return selected_utilization_value, selected_action

    def next_current_player(self):
        """ Metoda vrátí protihrače v procházení stavového prostoru
        """
        return 3 - self.current_player

    def next_player(self):
        """ Metoda vrátí protihráče
        """
        return 3 - self.player
    


# Partie piškvorek

In [3]:
# Vytvoření počátečního stavu partie
    # herní plán je prázdní
    # na tahu je hrač 1
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1, max_depth=3)

In [4]:
# Vypsání počátečního stavu
print(state.gameplan)

[[0 0 0]
 [0 0 0]
 [0 0 0]]


In [5]:
# Sledujte čas a počty generovaných stavů s postupující partií

# Cyklus pro hru, kde hrají proti sobě dvě kopie algoritmu
while True:
    # Kontrola, zda není partie u konce
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Kontrola, zda není remíza
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # tah hráče
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    state = state.expand(player_action)
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # přepnutí partie na druhého hráče
    state.player = state.next_player()

Player 1
Select action: (0, 0)
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Generated states 587.
Player 2
Select action: (0, 1)
[[1 2 0]
 [0 0 0]
 [0 0 0]]
Generated states 65.
Player 1
Select action: (0, 2)
[[1 2 1]
 [0 0 0]
 [0 0 0]]
Generated states 8.
Player 2
Select action: None


TypeError: 'NoneType' object is not subscriptable

# Úkol
- Do algoritmu přidejte omezeni na maximalní prohledávanou hloubku.
- Zkoušejte různé omezení prohledávání do hloubky.
- Sledujte, jak se mění časy a počty vygenerovaných stavů
- Mění se výsledky hry?

In [ ]:
# Vytvoření počátečního stavu partie
    # herní plán je prázdní
    # na tahu je hrač 1
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1, max_depth=2)
# Cyklus pro hru, kde hrají proti sobě dvě kopie algoritmu
while True:
    # Kontrola, zda není partie u konce
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Kontrola, zda není remíza
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # tah hráče
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    state = state.expand(player_action)
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # přepnutí partie na druhého hráče
    state.player = state.next_player()

Player 1
Select action: (0, 0)
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Generated states 269175.
Player 2
Select action: (1, 1)
[[1 0 0]
 [0 2 0]
 [0 0 0]]
Generated states 26853.
Player 1
Select action: (0, 1)
[[1 1 0]
 [0 2 0]
 [0 0 0]]
Generated states 2425.
Player 2
Select action: (0, 2)
[[1 1 2]
 [0 2 0]
 [0 0 0]]
Generated states 77.
Player 1
Select action: (2, 0)
[[1 1 2]
 [0 2 0]
 [1 0 0]]
Generated states 66.
Player 2
Select action: (1, 0)
[[1 1 2]
 [2 2 0]
 [1 0 0]]
Generated states 17.
Player 1
Select action: (1, 2)
[[1 1 2]
 [2 2 1]
 [1 0 0]]
Generated states 10.
Player 2
Select action: (2, 1)
[[1 1 2]
 [2 2 1]
 [1 2 0]]
Generated states 5.
Player 1
Select action: (2, 2)
[[1 1 2]
 [2 2 1]
 [1 2 1]]
Generated states 2.
Drawn
